In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/21Spring/COMS\ 6998/Project

/content/drive/MyDrive/21Spring/COMS 6998/Project


In [ ]:
%cd /content/drive/MyDrive/21Spring/COMS\ 6998/Project/6998-sec11-final-project

/content/drive/MyDrive/21Spring/COMS 6998/Project/6998-sec11-final-project


In [ ]:
%cd /content/drive/MyDrive/21Spring/COMS\ 6998/Project/6998-sec11-final-project-sys

/content/drive/MyDrive/21Spring/COMS 6998/Project/6998-sec11-final-project-sys


In [ ]:
%cd /content/drive/MyDrive/21Spring/COMS\ 6998/Project/Final_Project

/content/drive/MyDrive/21Spring/COMS 6998/Project/Final_Project


In [ ]:
from vehicle_signature.contrastive import ContrastiveLoss
from vehicle_signature.siamese import SiameseNetwork
import torch
import numpy as np

In [ ]:
from vehicle_detection import detector
class LoadModelTF:
  @classmethod
  def load_model(cls, model_path):
    model = detector.get_detector()
    model.predict(np.random.randn(1, 150, 150, 3))
    model.load_weights(model_path)
    return model

In [ ]:
model_keras = LoadModelTF.load_model("logs/vehicle_detection/simple_cnn/ckpt")
model_keras.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 150, 150, 16)      2368      
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 75, 75, 32)        4640      
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 75, 75, 64)        18496     
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 38, 38, 128)       73856     
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 38, 38, 256)       295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 19, 19, 256)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 92416)            

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils
import matplotlib.pyplot as plt
import torchvision.utils
from functools import reduce
from operator import __add__

def compute_padding(k):
    return (k - 1) // 2

class TorchDetector(nn.Module):
    def __init__(self):
        super(TorchDetector, self).__init__()

        # Setting up the Sequential of CNN Layers
        self.cnn1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=7, stride=1, padding=compute_padding(7)),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=compute_padding(3)),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=compute_padding(3)),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=compute_padding(3)),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=compute_padding(3)),
            nn.MaxPool2d(2, stride=2)
        )

        # Defining the fully connected layers
        self.fc1 = nn.Sequential(
            nn.Linear(92416, 20),
            nn.ReLU(inplace=True),
            nn.Linear(20, 1), 
            nn.Sigmoid()
        )

    def forward(self, x):
        # Forward pass
        output = self.cnn1(x)
        print(output.shape)
        output = output.view(output.size(0), -1)
        print(output.shape)
        output = self.fc1(output)
        return output

In [ ]:
weights = model_keras.get_weights()
torchdetector = TorchDetector()

In [ ]:
torchdetector.cnn1

Sequential(
  (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (1): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [ ]:
torchdetector.fc1

Sequential(
  (0): Linear(in_features=92416, out_features=20, bias=True)
  (1): ReLU(inplace=True)
  (2): Linear(in_features=20, out_features=1, bias=True)
  (3): Sigmoid()
)

In [ ]:
torchdetector.cnn1[0].weight.data=torch.from_numpy(np.transpose(weights[0]))
torchdetector.cnn1[0].bias.data=torch.from_numpy(weights[1])
torchdetector.cnn1[1].weight.data=torch.from_numpy(np.transpose(weights[2]))
torchdetector.cnn1[1].bias.data=torch.from_numpy(weights[3])
torchdetector.cnn1[2].weight.data=torch.from_numpy(np.transpose(weights[4]))
torchdetector.cnn1[2].bias.data=torch.from_numpy(weights[5])
torchdetector.cnn1[3].weight.data=torch.from_numpy(np.transpose(weights[6]))
torchdetector.cnn1[3].bias.data=torch.from_numpy(weights[7])
torchdetector.cnn1[4].weight.data=torch.from_numpy(np.transpose(weights[8]))
torchdetector.cnn1[4].bias.data=torch.from_numpy(weights[9])
torchdetector.fc1[0].weight.data=torch.from_numpy(np.transpose(weights[10]))
torchdetector.fc1[0].bias.data=torch.from_numpy(weights[11])
torchdetector.fc1[2].weight.data=torch.from_numpy(np.transpose(weights[12]))
torchdetector.fc1[2].bias.data=torch.from_numpy(weights[13])

In [ ]:
torch.save(torchdetector.state_dict(), "torch_detector.pt")